<a href="https://colab.research.google.com/github/kridtapon/WFO-KVO-MACD/blob/main/WFO_KVO%2BMACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninsta

In [ ]:
import itertools
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt

# Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df

# Calculate MACD
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['MACD'] = df['Close'].ewm(span=fast_period, min_periods=fast_period).mean() - \
                 df['Close'].ewm(span=slow_period, min_periods=slow_period).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_period, min_periods=signal_period).mean()
    return df

# Walk-forward optimization with dynamic parameter ranges
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    short_period_range = range(10, 50, 5)
    long_period_range = range(30, 100, 5)
    signal_period_range = range(5, 20, 5)
    fast_period_range = range(5, 20, 5)
    slow_period_range = range(20, 50, 5)
    macd_signal_range = range(5, 15, 2)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Use itertools to loop through all combinations dynamically
        for params in itertools.product(
            short_period_range,
            long_period_range,
            signal_period_range,
            fast_period_range,
            slow_period_range,
            macd_signal_range,
        ):
            short_period, long_period, signal_period, fast_period, slow_period, macd_signal = params

            # Ensure valid parameter relationships
            if short_period >= long_period or fast_period >= slow_period:
                continue

            # Calculate indicators on the training data
            train_data = klinger_volume_oscillator(train_data.copy(), short_period, long_period, signal_period)
            train_data = calculate_macd(train_data, fast_period, slow_period, macd_signal)

            # Generate entry and exit signals
            entries = (
                (train_data['KVO'] > train_data['KVO_Signal']) &
                (train_data['MACD'] > train_data['MACD_Signal'])
            )
            exits = (
                (train_data['KVO'] < train_data['KVO_Signal']) &
                (train_data['MACD'] < train_data['MACD_Signal'])
            )

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (short_period, long_period, signal_period, fast_period, slow_period, macd_signal)

        # Test with the best parameters on the test data
        test_data = klinger_volume_oscillator(test_data.copy(), *best_params[:3])
        test_data = calculate_macd(test_data, *best_params[3:])

        entries = (
            (test_data['KVO'] > test_data['KVO_Signal']) &
            (test_data['MACD'] > test_data['MACD_Signal'])
        )
        exits = (
            (test_data['KVO'] < test_data['KVO_Signal']) &
            (test_data['MACD'] < test_data['MACD_Signal'])
        )

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params,
            'Test_Return': portfolio.total_return()
        })

        # Print the performance for the year
        print(f"Year {test_year}: Walk-Forward Optimization Return = {portfolio.total_return():.2f}")

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2010-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2024)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)


[*********************100%***********************]  1 of 1 completed


In [15]:
# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    yearly_data = klinger_volume_oscillator(yearly_data, *params[:3])
    yearly_data = calculate_macd(yearly_data, *params[3:])

    entries = (yearly_data['KVO'] > yearly_data['KVO_Signal']) & \
              (yearly_data['MACD'] > yearly_data['MACD_Signal'])
    exits = (yearly_data['KVO'] < yearly_data['KVO_Signal']) & \
            (yearly_data['MACD'] < yearly_data['MACD_Signal'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for test only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest combined strategy
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display stats and plot
print(portfolio.stats())
portfolio.plot().show()

<ipython-input-13-d9cfadb57e40>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-d9cfadb57e40>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-d9cfadb57e40>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-d9cf

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           376378.848522
Total Return [%]                       276.378849
Benchmark Return [%]                   104.995855
Max Gross Exposure [%]                      100.0
Total Fees Paid                      10149.418514
Max Drawdown [%]                        36.506123
Max Drawdown Duration                       545.0
Total Trades                                   25
Total Closed Trades                            25
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 52.0
Best Trade [%]                            38.9557
Worst Trade [%]                        -19.972302
Avg Winning Trade [%]                   16.895655
Avg Losing Trade [%]                     -4.88042


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df['KVO'], df['KVO_Signal']

# Calculate MACD
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['MACD'] = df['Close'].ewm(span=fast_period, min_periods=fast_period).mean() - \
                 df['Close'].ewm(span=slow_period, min_periods=slow_period).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_period, min_periods=signal_period).mean()
    return df['MACD'], df['MACD_Signal']

# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate KVO and MACD
df['KVO'], df['KVO_Signal'] = klinger_volume_oscillator(df, 40, 80, 14)
df['MACD'], df['MACD_Signal'] = calculate_macd(df, 14, 29, 9)

# Define entry and exit signals based on KVO and MACD
df['Entry'] = (
    (df['KVO'] > df['KVO_Signal']) &  # KVO crosses above KVO Signal
    (df['MACD'] > df['MACD_Signal']) &  # MACD crosses above Signal line
    (df['KVO'].shift(1) <= df['KVO_Signal'].shift(1))  # Previous KVO was below Signal
)

df['Exit'] = (
    (df['KVO'] < df['KVO_Signal']) &  # KVO crosses below KVO Signal
    (df['MACD'] < df['MACD_Signal']) &  # MACD crosses below Signal line
    (df['KVO'].shift(1) >= df['KVO_Signal'].shift(1))  # Previous KVO was above Signal
)

# Filter data for test only
df = df[(df.index.year >= 2024) & (df.index.year <= 2025)]

# Convert signals to boolean arrays
entries = df['Entry'].to_numpy()
exits = df['Exit'].to_numpy()

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()